# RossmannStoreSales

## Dataset Information

### train.csv
This is the main training dataset which contains data about the sales figures for a store on a
particular date.

#### Data Fields:
1. Store: a unique numerical store identifier (1 - 1,115)
2. DayOfWeek: the day of week (1 - 7)
3. Date: the date, ranging from 2013-01-01 to 2015-07-31
4. Sales: the turnover of the specified store on the specified date
5. Customers: the number of customers of the specified store on the specified date
6. Open: indicates whether the store was open (0 = Closed, 1 = Open)
7. Promo: indicates whether the store was running a promotion (0 = No, 1 = Yes)
8. StateHoliday: indicates if it was a state holiday (a = Public holiday, b = Easter holiday, c = Christmas, 0 = None)
9. SchoolHoliday: indicates if it was a school holiday (0 = No, 1 = Yes)

**Number of Records**: 1,017,209

### store.csv
This dataset contains supplementary information about each of the 1,115 stores and helps
identify unique features which may (or may not) affect sales.

#### Data Fields
1. Store: a unique numerical store identifier (1 - 1,115)
2. StoreType: differentiates between the 4 different types of stores (a, b, c, d)
3. Assortment: describes the assortment of goods carried by the store (a = Basic, b = Extra, c = Extended)
4. CompetitionDistance: the distance (in metres) to the nearest competitor’s store
5. CompetitionOpenSinceMonth: the month in which the competition opened
6. CompetitionOpenSinceYear: the year in which the competition opened
7. Promo2: indicates if a store is participating in a continuing and consecutive promotion (0 = No, 1 = Yes)
8. Promo2SinceWeek: the week of the year in which the store began participating in Promo2 (from 1 - 52, presumably, but some weeks are unrepresented in the data)
9. Promo2SinceYear: the year in which the store began participating in Promo2 (from 2009 - 2015)
10. PromoInterval: describes the consecutive intervals in which Promo2 is activated, giving the months the promotion is renewed (either “Jan,Apr,Jul,Oct”, “Feb,May,Aug,Nov” or “Mar,Jun,Sept,Dec”)

**Number of Records**: 1,115

### test.csv
This dataset is to be used for testing and evaluating the model.

#### Data Fields
Same as train.csv, with the exclusion of Customers & Sales (Sales to be predicted by the model) and an additional field Id which represents a (Store, Date) tuple that is used to label predictions for submission to Kaggle.

**Number of Records**: 41,088